In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Activation, Dropout, Flatten
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from typing import List
from pathlib import Path

In [2]:
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


In [ ]:
# get the data
filename = 'dataset/emotions_dataset.csv'
df = pd.read_csv(filename, na_filter=False)

In [ ]:
df_train = df[(df.usage == 'Training') | (df.usage == 'PublicTest')].copy().reset_index()
df_train['labels'] = [label_map[e] for e in df_train.emotion]

df_test = df[df.usage == 'PrivateTest'].copy().reset_index()
df_test['labels'] = [label_map[e] for e in df_test.emotion]
del df


In [ ]:
def pixels_to_images(pixels_series: pd.Series) -> np.ndarray:
    pixels_lists = [[int(n) for n in p.split()] for p in pixels_series]
    X = np.array(pixels_lists) / 255
    X = X.reshape(-1, 48, 48, 1)
    return X

In [ ]:
npy_data = 'dataset/numpy_images.npy'
if Path(npy_data).is_file():
    with open('dataset/numpy_images.npy', 'rb') as f:
        X = np.load(f)
else:
    X = pixels_to_images(df_train.pixels)
    with open('dataset/numpy_images.npy', 'wb') as f:
        np.save(f, X)

X_test = pixels_to_images(df_test.pixels)

Y = df_train.emotion.astype(int).to_list()
y_test = df_test.emotion.astype(int).to_list()

print(X.shape)

In [ ]:
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 5
true_labels = []
preds = []
for i in range(1, columns*rows +1):
    img_id = np.random.choice(X.shape[0])
    img = X[img_id, :, :, 0]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, cmap='gray')
    plt.xticks([], [])
    plt.yticks([], [])
    true_labels.append(df_train.labels.iloc[img_id])
    preds.append(true_labels[i-1])
    plt.title(f"{true_labels[i-1]}")
fig.tight_layout()
plt.show()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=0)

In [ ]:
X_train.shape

In [ ]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    optim = tf.keras.optimizers.Adam(learning_rate=2e-4)
    
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=optim)
    
    return model
model=my_model()
# model.summary()

In [ ]:
image_generator = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.1,  # set range for random shear
    zoom_range=0.0,  # set range for random zoom
    channel_shift_range=0.0,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode="nearest",
    cval=0.0,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format="channels_last",
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0,
)
image_generator = image_generator.flow(X_train, y_train, batch_size=64)

In [ ]:
val_generator = ImageDataGenerator()
val_generator = val_generator.flow(X_val, y_val, batch_size=64)

In [ ]:
path_model='checkpoints/{epoch:02d}-{val_loss:.6f}.hdf5'
Path('checkpoints').mkdir(parents=True, exist_ok=True)
h = model.fit(
    x=image_generator, 
    steps_per_epoch=len(X_train) // 64,
    epochs=100, 
    verbose=1, 
    validation_data=val_generator,
    shuffle=True,
    callbacks=[
        ModelCheckpoint(filepath=path_model),
        EarlyStopping(patience=15),
        ReduceLROnPlateau(patience=6, factor=0.3)
    ]
)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 5
true_labels = []
preds = []
preds = model.predict(X_test)
for i in range(1, columns*rows +1):
    img_id = np.random.choice(X_test.shape[0])
    img = X_test[img_id, :, :, 0]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, cmap='gray')
    plt.xticks([], [])
    plt.yticks([], [])
    plt.title(f"{label_map[y_test[img_id]]}, {label_map[np.argmax(preds[img_id])]}")
fig.tight_layout()
plt.show()

In [3]:
model = load_model('artifacts/34-0.953775.hdf5')

prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNet(prototxtPath, weightsPath)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
image = cv2.imread('dataset/example_images/surprised.jpg')
orig = image.copy()
(h, w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

net.setInput(blob)
detections = net.forward()

# loop over the detections
for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        height = endY - startY
        
        midX = int((endX + startX) / 2)
        # extract the face ROI, convert it from BGR to RGB channel
        # ordering, resize it to 224x224, and preprocess it
        face = image[startY:endY, (midX - int(height / 2)):(midX + int(height / 2))]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face, (48, 48))
        face = img_to_array(face)
        face = np.expand_dims(face, axis=0)

        # pass the face through the model to determine if the face
        # has a mask or not
        preds = model.predict(face)[0]

        # determine the class label and color we'll use to draw
        # the bounding box and text
        color = (0, 255, 0)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label_map[np.argmax(preds)], max(preds) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

cv2.imshow("Output", image)
while True:
    key = cv2.waitKey(0)
    if key in [27, ord('q'), ord('Q')]:
        cv2.destroyAllWindows()
        break

In [ ]:
plt.imshow(np.squeeze(face[0,:,:,:]), cmap='gray')

In [ ]:
def emotion_analysis(emotions):
    y_pos = np.arange(len(label_map))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, label_map)
    plt.ylabel('percentage')
    plt.title('emotion')
    
plt.show()

In [ ]:
y_pred=model.predict(X_test)
#print(y_pred)
y_test.shape

In [ ]:
#import seaborn as sn
#import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
#from sklearn.metrics import confusion_matrix
#%matplotlib inline
#cm = confusion_matrix(np.where(y_test == 1)[1], y_pred)
#cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#df_cm = pd.DataFrame(cm, index = [i for i in "0123456"],
                  #columns = [i for i in "0123456"])
#plt.figure(figsize = (20,15))
#sn.heatmap(df_cm, annot=True)

**Real Time Expression Prediction**

In [ ]:
from skimage import io
img = image.load_img('../input/myimage/Shawon.jpg', grayscale=True, target_size=(48, 48))
show_img = image.load_img('../input/myimage/Shawon.jpg', grayscale=False, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
#print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])
        

**Live Demo of Production Level Project**

[Facial Expression Detection Web App](https://faceai.herokuapp.com/)